# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as this task requires statistical modeling and machine learning expertise.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
    "context": "The task involves analyzing market data for the selected stock (AAPL) to identify trends and predict market movements. Statistical modeling and machine learning techniques will be used to extract insights and predict future market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
I need to gather

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Short-term Momentum Trading:
- Potential Risks:
    - Market volatility leading to unexpected price fluctuations.
    - Incorrect identification of entry and exit points.
    - Stop-loss orders not being triggered due to rapid price movements.
    - Inadequate review and adjustment of the strategy based on changing market conditions.
- Mitigation Strategies:
    - Implement robust risk management practices to protect against market volatility.
    - Conduct thorough technical analysis to accurately identify entry and exit points.
    - Regularly monitor and adjust stop-loss orders to manage risk effectively.
    - Stay informed about market conditions and be prepared to adapt the strategy as needed.

Options Trading:
- Potential Risks:
    - Incorrect analysis leading to wrong options purchase decisions.
    - Inaccurate timing for entering and exiting options positions.
    - High volatility in options pricing and volume.
    - Inadequate use of risk management strategies like hedging.
- Mitigation Strategies:
    - Conduct detailed analysis to ensure accurate options purchase decisions.
    - Monitor options pricing and volume closely for optimal entry and exit points.
    - Utilize effective risk management techniques such as hedging to protect against losses.
    - Stay updated on options market trends and adjust strategies accordingly.

Swing Trading:
- Potential Risks:
    - Failure to identify profitable swing trading opportunities.
    - Inaccurate profit targets and stop-loss levels.
    - Ineffective use of technical analysis indicators.
    - Lack of regular review and adjustment of the strategy.
- Mitigation Strategies:
    - Conduct thorough analysis to identify potential swing trading opportunities.
    - Set clear and achievable profit targets and stop-loss levels.
    - Utilize technical analysis indicators effectively for timing entry and exit points.
    - Regularly review trade performance and adjust the strategy as needed.

Sector Rotation:
- Potential Risks:
    - Failure to accurately monitor economic cycles and sector trends.
    - Ineffective shift of investments between sectors.
    - Incorrect assessment of growth potential and risks within different sectors.
    - Lack of discipline in sector rotation approach.
- Mitigation Strategies:
    - Monitor economic cycles and sector-specific trends closely.
    - Strategically shift investments between sectors based on market conditions.
    - Conduct thorough fundamental analysis to assess growth potential and risks.
    - Implement a disciplined approach to sector rotation to capitalize on opportunities.

In conclusion, it is essential to carefully evaluate and mitigate the risks associated with each trading strategy for AAPL to ensure alignment with the user's risk tolerance and trading preferences. Regular monitoring, analysis, and adjustment of the strategies are crucial for successful trading activities.